In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
# Load your dataset
# Assuming the dataset is a CSV file with 'text' and 'emotion' columns
data = pd.read_csv('text.csv')


In [ ]:
data.head()

#Yt@tech8445

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
data['label'] = data['label'].replace(0,'Sadness')
data['label'] = data['label'].replace(1,'Joy')
data['label'] = data['label'].replace(2,'Love')
data['label'] = data['label'].replace(3,'Anger')
data['label'] = data['label'].replace(4,'Fear')
data['label'] = data['label'].replace(5,'Surprise')

In [ ]:
data.rename(columns={'label': 'emotion'}, inplace=True)

In [ ]:
data.head()

,Unnamed: 0,text,emotion
0,0,i just feel really helpless and heavy hearted,Fear
1,1,ive enjoyed being able to slouch about relax a...,Sadness
2,2,i gave up my internship with the dmrg and am f...,Fear
3,3,i dont know i feel so lost,Sadness
4,4,i am a kindergarten teacher and i am thoroughl...,Fear


In [ ]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,text,emotion
0,i just feel really helpless and heavy hearted,Fear
1,ive enjoyed being able to slouch about relax a...,Sadness
2,i gave up my internship with the dmrg and am f...,Fear
3,i dont know i feel so lost,Sadness
4,i am a kindergarten teacher and i am thoroughl...,Fear


In [ ]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
padded_sequences = pad_sequences(sequences, padding='post')

In [ ]:
# Convert labels to integers
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(data['emotion'])
label_sequences = np.array(label_tokenizer.texts_to_sequences(data['emotion']))


In [ ]:
data.head()

,text,emotion
0,i just feel really helpless and heavy hearted,Fear
1,ive enjoyed being able to slouch about relax a...,Sadness
2,i gave up my internship with the dmrg and am f...,Fear
3,i dont know i feel so lost,Sadness
4,i am a kindergarten teacher and i am thoroughl...,Fear


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, label_sequences, test_size=0.2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=178),  # Reduced embedding dimension
    Bidirectional(LSTM(32, return_sequences=True)),  # Reduced number of units
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(16, activation='relu'),  # Reduced dense layer units
    Dense(len(label_tokenizer.word_index) + 1, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)  # Reduced epochs and adjusted batch size



Epoch 1/5
5211/5211 ━━━━━━━━━━━━━━━━━━━━ 1827s 351ms/step - accuracy: 0.8937 - loss: 0.2894 - val_accuracy: 0.9372 - val_loss: 0.1039
Epoch 2/5
5211/5211 ━━━━━━━━━━━━━━━━━━━━ 1876s 349ms/step - accuracy: 0.9396 - loss: 0.0999 - val_accuracy: 0.9392 - val_loss: 0.0944
Epoch 3/5
5211/5211 ━━━━━━━━━━━━━━━━━━━━ 1836s 352ms/step - accuracy: 0.9415 - loss: 0.0924 - val_accuracy: 0.9399 - val_loss: 0.0913
Epoch 4/5
5211/5211 ━━━━━━━━━━━━━━━━━━━━ 1844s 354ms/step - accuracy: 0.9421 - loss: 0.0888 - val_accuracy: 0.9403 - val_loss: 0.0918
Epoch 5/5
5211/5211 ━━━━━━━━━━━━━━━━━━━━ 1943s 366ms/step - accuracy: 0.9436 - loss: 0.0862 - val_accuracy: 0.9395 - val_loss: 0.0913
